In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib

In [ ]:
import dolfinx as dfx

import matplotlib.pyplot as plt

from mpi4py.MPI import COMM_WORLD as comm

import numpy as np

import pyvista as pv

import ufl

from pyMoBiMP.cahn_hilliard_utils import (
    c_of_y, compute_chemical_potential, _free_energy as free_energy)

from pyMoBiMP.fenicsx_utils import time_stepping

In [ ]:
n_rad = 64
n_part = 12

radial_grid = np.linspace(0, 1, n_rad)
particle_grid = np.linspace(0, 1, n_part)

rr, pp = np.meshgrid(radial_grid, particle_grid)

coords_grid = np.stack((rr, pp)).transpose((-1, 1, 0)).copy()

coords_grid.shape

coords_grid_flat = coords_grid.reshape(-1, 2).copy()

In [ ]:
# All the radial connections
elements_radial = [[[n_part * i + k, n_part * (i + 1) + k] for i in range(n_rad - 1)] 
                     for k in range(n_part)]

elements_radial = np.array(elements_radial).reshape(-1, 2)

# Connections between particles
elements_bc = (n_rad -1) * n_part + np.array([[k, k + 1] for k in range(n_part - 1)])
# elements_bc = []  # With elements at the outer edge the integration fails.

elements = np.array(list(elements_bc) + list(elements_radial))

In [ ]:
plt.figure()

for element in elements:
    # print(coords_grid_flat[element, :], coords_grid_flat[element, :])
    plt.plot(coords_grid_flat[element, 0], coords_grid_flat[element, 1], 'b')

plt.scatter(coords_grid_flat[:, 0], coords_grid_flat[:, 1])

plt.show()

In [ ]:
gdim = 2
shape = "interval"
degree = 1

cell = ufl.Cell(shape, geometric_dimension=gdim)
domain = ufl.Mesh(ufl.VectorElement("Lagrange", cell, degree))

mesh = dfx.mesh.create_mesh(comm, elements[:, :2], coords_grid_flat, domain)

In [ ]:
V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

u = dfx.fem.Function(V)
u.interpolate(lambda x: x[0] + x[1])

def plot_solution_on_grid(u):

    V = u.function_space

    topology, cell_types, x = dfx.plot.vtk_mesh(V)
    grid = pv.UnstructuredGrid(topology, cell_types, x)

    grid['u'] = u.x.array

    plotter = pv.Plotter()
    
    warped = grid.warp_by_scalar('u')

    plotter.add_mesh(warped, show_edges=True, show_vertices=False, )
    plotter.add_axes()

    plotter.show()

plot_solution_on_grid(u)

## Define an integral measure on the particle surface

In [ ]:
fdim = mesh.topology.dim - 1

facets = dfx.mesh.locate_entities(mesh, fdim, lambda x: np.isclose(x[0], 1.))

facet_markers = np.full_like(facets, 1)

facet_tag = dfx.mesh.meshtags(mesh, fdim, facets, facet_markers)

dA = ufl.Measure("ds", domain=mesh, subdomain_data=facet_tag)
dA_R = dA(1)

## Do some checks on the integral measures

In [ ]:
form = dfx.fem.form(dfx.fem.Constant(mesh, 1.) * ufl.dx)
value  = dfx.fem.assemble_scalar(form)

print(value)

form = dfx.fem.form(dfx.fem.Constant(mesh, 1.) * dA)
value  = dfx.fem.assemble_scalar(form)

print(value)

form = dfx.fem.form(dfx.fem.Constant(mesh, 1.) * dA_R)
value  = dfx.fem.assemble_scalar(form)

print(value)

## Define a PDE problem in the grid

### Elliptic problem

In [ ]:
elem1 = ufl.FiniteElement("Lagrange", mesh.ufl_cell(), 1)

V = dfx.fem.FunctionSpace(mesh, elem1)

In [ ]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

F = u.dx(0) * v.dx(0) * ufl.dx + 2 * v * ufl.dx

from dolfinx.fem.petsc import LinearProblem

In [ ]:
V_ex = dfx.fem.FunctionSpace(mesh, ("Lagrange", 2))
u_ex = dfx.fem.Function(V_ex)
u_ex.interpolate(lambda x: x[0]**2 + x[1])

u_bc = dfx.fem.Function(V)
u_bc.interpolate(u_ex)

dofs = dfx.fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 1.))
bc = dfx.fem.dirichletbc(u_bc, dofs)

In [ ]:
problem = LinearProblem(ufl.lhs(F), ufl.rhs(F), [bc])

u_ = problem.solve()

plot_solution_on_grid(u_)

In [ ]:
error = dfx.fem.assemble_scalar(dfx.fem.form((u_ - u_ex)**2 * ufl.dx))

print(error)

### Cahn-Hilliard Problem (uncoupled)

In [ ]:
elem1 = ufl.FiniteElement("Lagrange", mesh.ufl_cell(), 1)

V = dfx.fem.FunctionSpace(mesh, elem1 * elem1)

In [ ]:
u = dfx.fem.Function(V)
u0 = dfx.fem.Function(V)

y, mu = ufl.split(u)
y0, mu0 = ufl.split(u0)

v_c, v_mu = ufl.TestFunctions(V)

I_charge = dfx.fem.Constant(mesh, 1e-1)
theta = 1.0
dt = dfx.fem.Constant(mesh, 1e-6)

c = c_of_y(y)

V0, dofs = V.sub(0).collapse()
r = dfx.fem.Function(V0)
r.interpolate(lambda x: x[0])

def M(c):
    return c * (1 - c)

lam = 0.1

def grad_c_bc(c):
    return 0.

s_V = 4 * np.pi * r**2
s_A = 2 * np.pi * r**2

dx = ufl.dx  # The volume element

mu_chem = compute_chemical_potential(free_energy, c)
mu_theta = theta * mu + (theta - 1.0) * mu0

flux = M(c) * mu_theta.dx(0)

F1 = s_V * (c_of_y(y) - c_of_y(y0)) * v_mu * dx
F1 += s_V * flux * v_mu.dx(0) * dt * dx
F1 -= I_charge * v_mu * dt * dA_R

F2 = s_V * mu * v_c * dx
F2 -= s_V * mu_chem * v_c * dx
F2 -= lam * (s_V * c.dx(0) * v_c.dx(0) * dx)
F2 += grad_c_bc(c) * (s_A * v_c * dA_R)

F = F1 + F2

residual = dfx.fem.form(F)

from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver

problem = NonlinearProblem(F, u)
solver = NewtonSolver(comm, problem)

In [ ]:
# Do a single step with dt = 0 to propagate the initial data of y0 into y and mu
u0.sub(0).x.array[:] = -6  # This corresponds to roughly c = 1e-3

dt.value = 0.  # This enforces the first equation to be y = y0 and solves for mu.

print(dfx.fem.assemble_vector(residual).norm())
solver.solve(u)
print(dfx.fem.assemble_vector(residual).norm())

dt.value = 1.0
print(dfx.fem.assemble_vector(residual).norm())
vec = dfx.fem.assemble_vector(dfx.fem.form(s_A * v_mu * dA))
print(vec.norm())

print(dfx.fem.assemble_vector(dfx.fem.form(s_A * I_charge * v_mu * dA)).norm())

c = dfx.fem.Function(u.sub(0).collapse().function_space)
c.interpolate(
    dfx.fem.Expression(c_of_y(y), c.function_space.element.interpolation_points()))

plot_solution_on_grid(c)

In [ ]:
I_charge.value = 1.0

dt_min = 1e-9
dt_max = 1e-3

dt.value = 1e-8

T_final = 1.0
tol = 1e-4

u.x.scatter_forward()
u0.x.scatter_forward()

iterations, success = solver.solve(u)
print(iterations, success)

time_stepping(
    solver,
    u,
    u0,
    T_final,
    dt,
    dt_max=dt_max,
    dt_min=dt_min,
    tol=tol,
)

In [ ]:
y = u.sub(0)

c.interpolate(
    dfx.fem.Expression(c_of_y(y), c.function_space.element.interpolation_points()))

plot_solution_on_grid(c)

### Cahn-Hilliard (coupled)

In [ ]:
# particle parameters
V0, _ = V.sub(0).collapse()

Rs = dfx.fem.Function(V0)
Rs.x.array[:] = 1.

As = 4 * np.pi * Rs**2

L_mean = 10.
L_var_rel = 0.1

Ls = dfx.fem.Function(V0)
Ls.x.array[:] = 1 + L_var_rel * (2 * np.random.random(Ls.x.array.shape) - 1)

A_ufl = As * dA_R
A = dfx.fem.assemble_scalar(dfx.fem.form(A_ufl))

a_ratios = As / A

L_ufl = a_ratios * Ls * dA_R
L = dfx.fem.assemble_scalar(dfx.fem.form(L_ufl))

In [ ]:
u = dfx.fem.Function(V)
u0 = dfx.fem.Function(V)

y, mu = ufl.split(u)
y0, mu0 = ufl.split(u0)

v_c, v_mu = ufl.TestFunctions(V)

I_global = dfx.fem.Constant(mesh, 1e-1)

I_particle = dfx.fem.Function(V0)
OCP = dfx.fem.Function(V0)

OCP_expr = dfx.fem.Expression(- Ls / L * a_ratios * mu, 
                              V0.element.interpolation_points())

V_cell_form = dfx.fem.form(- (I_global / L - OCP) * dA_R)

def callback():

    OCP.interpolate(OCP_expr)

    V_cell = dfx.fem.assemble_scalar(V_cell_form)

    mu = u.sub(1).collapse()

    I_particle.x.array[:] = - Ls.x.array * (mu.x.array + V_cell)

theta = 1.0
dt = dfx.fem.Constant(mesh, 1e-6)

c = c_of_y(y)

V0, dofs = V.sub(0).collapse()
r = dfx.fem.Function(V0)
r.interpolate(lambda x: x[0])

def M(c):
    return c * (1 - c)

lam = 0.1

def grad_c_bc(c):
    return 0.

s_V = 4 * np.pi * r**2
s_A = 2 * np.pi * r**2

dx = ufl.dx  # The volume element

mu_chem = compute_chemical_potential(free_energy, c)
mu_theta = theta * mu + (theta - 1.0) * mu0

flux = M(c) * mu_theta.dx(0)

F1 = s_V * (c_of_y(y) - c_of_y(y0)) * v_mu * dx
F1 += s_V * flux * v_mu.dx(0) * dt * dx
F1 -= s_A * I_particle * v_mu * dt * dA_R

F2 = s_V * mu * v_c * dx
F2 -= s_V * mu_chem * v_c * dx
F2 -= lam * (s_V * c.dx(0) * v_c.dx(0) * dx)
F2 += grad_c_bc(c) * (s_A * v_c * dA_R)

F = F1 + F2

residual = dfx.fem.form(F)

NonlinearProblemBase = NonlinearProblem


class NonlinearProblem(NonlinearProblemBase):
    def form(self, x):
        callback()
        super().form(x)


problem = NonlinearProblem(F, u)
solver = NewtonSolver(comm, problem)

In [ ]:
# Do a single step with dt = 0 to propagate the initial data of y0 into y and mu
u0.sub(0).x.array[:] = -6  # This corresponds to roughly c = 1e-3

In [ ]:
dt_min = 1e-9
dt_max = 1e-3

dt.value = 1e-8

T_final = 1.0
tol = 1e-4

I_global.value = 0.01

u.x.scatter_forward()
u0.x.scatter_forward()

iterations, success = solver.solve(u)
print(iterations, success)

time_stepping(
    solver,
    u,
    u0,
    T_final,
    dt,
    dt_max=dt_max,
    dt_min=dt_min,
    dt_increase=1.01,
    tol=tol,
)

In [ ]:
y = u.sub(0)

c = dfx.fem.Function(V0)
c.interpolate(
    dfx.fem.Expression(c_of_y(y), V0.element.interpolation_points()))

plot_solution_on_grid(c)

## Parallel Mesh partition

In [ ]:
import ipyparallel as ipp
cluster = ipp.Cluster(engines="mpi", n=2)
rc = cluster.start_and_connect_sync()

In [ ]:
%%px
from mpi4py import MPI as MPIpx
import numpy as np
import ufl
import dolfinx as dfx

print(f"{MPIpx.COMM_WORLD.rank=} {MPIpx.COMM_WORLD.size=}")

In [ ]:
%%px

n_rad = 64
n_part = 12

if MPIpx.COMM_WORLD.rank == 0:

    radial_grid = np.linspace(0, 1, n_rad)
    particle_grid = np.linspace(0, 1, n_part)

    rr, pp = np.meshgrid(radial_grid, particle_grid)

    coords_grid = np.stack((rr, pp)).transpose((-1, 1, 0)).copy()

    coords_grid_flat = coords_grid.reshape(-1, 2).copy()

    # All the radial connections
    elements_radial = [[
        [n_part * i + k, n_part * (i + 1) + k] for i in range(n_rad - 1)] 
                        for k in range(n_part)]

    elements_radial = np.array(elements_radial).reshape(-1, 2)

    elements = elements_radial

else:
    coords_grid_flat = np.empty((0, 2), dtype=np.float64)
    elements = np.empty((0, 2), dtype=np.int64)


In [ ]:
%%px

gdim = 2
shape = "interval"
degree = 1

cell = ufl.Cell(shape, geometric_dimension=gdim)
domain = ufl.Mesh(ufl.VectorElement("Lagrange", cell, degree))

try:
    mesh = dfx.mesh.create_mesh(
        MPIpx.COMM_WORLD, elements[:, :2], coords_grid_flat, domain)
except Exception as e:
    print(e)

In [ ]:
%%px

V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

u = dfx.fem.Function(V)
u.interpolate(lambda x: x[0] + x[1])

def plot_solution_on_grid(u):

    V = u.function_space

    topology, cell_types, x = dfx.plot.vtk_mesh(V)
    grid = pv.UnstructuredGrid(topology, cell_types, x)

    grid['u'] = u.x.array

    plotter = pv.Plotter()

    warped = grid.warp_by_scalar('u')

    plotter.add_mesh(warped, show_edges=True, show_vertices=False, )
    plotter.add_axes()

    plotter.show()

if MPIpx.COMM_WORLD.rank == 0:
    plot_solution_on_grid(u)